In [6]:
import pandas as pd
import os

In [8]:
def nice_data(path, name):
    
    filename = f'shm_details_{name}.txt'
    filename = os.path.join(path, filename)
    # тут заголовки из 0 строчки файла будут
    header = None

    # тут будут лежать строчки для будущего датафрейма
    reads = []

    # тест на наличие доп. полей в строчках с названиями ридов
    additional_info = set()

    # текущая строчка тут будет
    new_read = []

    with open(filename) as f:

        # по каждой строке файла
        for line in f:

            # если у нас 0 строчка (header == None в этот момент)
            if not header:
                # то записываем в него названия колонок из файла
                header = line.strip().split('\t')

            # если у нас трочка с названием
            elif line.startswith('Read_name'):

                # будем сюда писать значения полей из строчки
                new_read_title = []

                # разбили ее по табам, но сейчас значения в массиве типа Read_length:358
                split_line = line.strip().split('\t')

                # поэтому пройдемся по ним циклом
                for part in split_line:

                    # разделим по :, получим, например ['Read_length', '358']
                    column, value = part.split(':')

                    # запишем в массив, где значения полей текушей последовательности
                    new_read_title.append(value)

                    # на всякий добавим поля из строки в сет, чтобы проверить, что мы учли все
                    # и внезапно не всплывет в середине файла что-то левое
                    additional_info.add(column)

            # если у нас строчка с данными
            else:

                # делим ее по табам
                new_read = line.strip().split('\t')

                # записываем в reads объединение массива с названием и со строчкой
                reads.append(new_read_title + new_read)
    # создаем датафрейм
    data = pd.DataFrame(data=reads, columns=['Read_name', 'Read_length', 'Gene_name', 
                                             'Gene_length', 'Segment', 'Chain_type'] + header)
    
    output_name = os.path.join(path, f'nice_data_{name}.csv')
    data.to_csv(output_name, index=False)
    
    # лень преобразовывать в инты, поэтому просто считаю, а пандас сам определит их интами)))
    data = pd.read_csv(output_name)
    
    # отдельная колонка для генов без аллелей
    data['Gene_no_allele'] = data.Gene_name.str.split('*').str[0]

In [16]:
name = 'plasma'
path = f'HW1_immuno/{name}'

In [17]:
nice_data(path, name)